In [ ]:
import pandas as pd
import warnings
import calendar
import matplotlib.pyplot as plt
import numpy as np

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Carregar os dados
file_path = r"C:/Kidy/PREDITIVA/DADOS_PREDITIVA.xlsx"
df = pd.read_excel(file_path, sheet_name='DADOS PREDITIVA')

# Garantir tipo datetime
df['Data Cadastro'] = pd.to_datetime(df['Data Cadastro'], errors='coerce')
df['Data Ultima Compra'] = pd.to_datetime(df['Data Ultima Compra'], errors='coerce')

def analisar_grupo_cliente(codigo_grupo_cliente):
    dados_grupo = df[df['Codigo Grupo Cliente'] == codigo_grupo_cliente].copy()

    if dados_grupo.empty:
        print(f"Nenhum dado encontrado para o Código Grupo Cliente: {codigo_grupo_cliente}")
        return
    
    nome_grupo = dados_grupo['Grupo Cliente'].iloc[0]
    ultima_data = dados_grupo['Data Ultima Compra'].max()
    ultima_compra = ultima_data.strftime('%d/%m/%Y') if pd.notnull(ultima_data) else 'Sem compras registradas'
    
    print("==========================")
    print(f"📌 Grupo Cliente: {nome_grupo}")
    print(f"🗓️  Última Compra: {ultima_compra}")
    print("==========================")
    
    if dados_grupo['Data Cadastro'].isnull().all():
        print("\n⚠️ Nenhuma data de cadastro disponível para análise!")
        return
    
    dados_grupo.loc[:, 'Mes Cadastro'] = dados_grupo['Data Cadastro'].dt.month
    dados_grupo.loc[:, 'Mes Pedido'] = dados_grupo['Data Cadastro'].dt.month
    
    if dados_grupo['Mes Pedido'].isnull().all():
        print("\n⚠️ Não há registros de mês de pedido para sugerir uma melhor época de oferta.")
    else:
        melhor_mes_num = dados_grupo['Mes Pedido'].mode()[0]
        melhor_mes_nome = calendar.month_name[melhor_mes_num]
        print(f"\n👉 Melhor mês para oferecer produtos ao grupo (frequência de pedidos): {melhor_mes_nome}")
    
    melhor_mes_volume = (dados_grupo.groupby('Mes Pedido')['Qtd Venda']
                         .sum()
                         .sort_values(ascending=False)
                         .head(1))
    
    if not melhor_mes_volume.empty:
        melhor_mes_num_volume = melhor_mes_volume.index[0]
        melhor_mes_nome_volume = calendar.month_name[melhor_mes_num_volume]
        print(f"📈 Mês com maior volume vendido: {melhor_mes_nome_volume} ({melhor_mes_volume.iloc[0]} unidades)")

    # Cálculo da probabilidade de compra:
    total_pedidos = dados_grupo['Data Cadastro'].nunique()

    probabilidade_ref = (dados_grupo.groupby(['Referencia', 'Linha'])
                         .agg({
                             'Data Cadastro': 'nunique',
                             'Qtd Venda': 'sum'
                         })
                         .rename(columns={
                             'Data Cadastro': 'Pedidos Distintos',
                             'Qtd Venda': 'Qtd Vendida'
                         })
                        )

    # Calculando a probabilidade como proporção dos pedidos distintos
    probabilidade_ref['Probabilidade Compra'] = probabilidade_ref['Pedidos Distintos'] / total_pedidos

    # Ordenando pela maior probabilidade
    top_probabilidade = probabilidade_ref.sort_values(by='Probabilidade Compra', ascending=False).head(10)

    # Agora, vamos preparar a tabela final com as colunas solicitadas
    # Separar as vendas no 1º e 2º semestre para essas referências

    dados_grupo['Semestre'] = dados_grupo['Mes Pedido'].apply(lambda x: '1º Semestre' if x <= 6 else '2º Semestre')

    semestre_vendas = (dados_grupo[dados_grupo['Referencia'].isin(top_probabilidade.index.get_level_values(0))]
                       .groupby(['Referencia', 'Semestre'])['Qtd Venda']
                       .sum()
                       .unstack(fill_value=0))

    # Se faltar alguma coluna de semestre, garante que ela exista
    if '1º Semestre' not in semestre_vendas.columns:
        semestre_vendas['1º Semestre'] = 0
    if '2º Semestre' not in semestre_vendas.columns:
        semestre_vendas['2º Semestre'] = 0

    # Juntar as infos de probabilidade e semestre
    top_probabilidade = top_probabilidade.merge(semestre_vendas, on='Referencia', how='left')

    # Reorganizar o dataframe para exibição
    top_probabilidade_display = top_probabilidade.reset_index()[['Linha', 'Referencia', 'Probabilidade Compra', '1º Semestre', '2º Semestre']]

    # Adicionar numeração
    top_probabilidade_display.index = top_probabilidade_display.index + 1
    top_probabilidade_display.index.name = 'Nº'

    print("\n👉 Top 10 Referências com Maior Probabilidade de Compra:")
    print(top_probabilidade_display.to_string(index=True, float_format=lambda x: f'{x:.2%}' if isinstance(x, float) and x <= 1 else f'{x:.0f}'))

    # ==========================
    # GRÁFICO COM PROBABILIDADE
    # ==========================
    linhas = top_probabilidade_display['Linha']
    primeiro_semestre = top_probabilidade_display['1º Semestre']
    segundo_semestre = top_probabilidade_display['2º Semestre']

    x = np.arange(len(linhas))  # posição no eixo X
    largura = 0.35  # largura das barras

    fig, ax = plt.subplots(figsize=(12, 6))

    barras1 = ax.bar(x - largura/2, primeiro_semestre, largura, label='1º Semestre')
    barras2 = ax.bar(x + largura/2, segundo_semestre, largura, label='2º Semestre')

    ax.set_xlabel('Linhas de Produto')
    ax.set_ylabel('Quantidade Vendida')
    ax.set_title('Top 10 Referências com Maior Probabilidade de Compra (por Linha e Semestre)')
    ax.set_xticks(x)
    ax.set_xticklabels(linhas, rotation=45, ha='right')
    ax.legend()

    # Exibir os valores em cima das barras
    def adicionar_rotulos(barras):
        for barra in barras:
            altura = barra.get_height()
            ax.annotate(f'{int(altura)}',
                        xy=(barra.get_x() + barra.get_width() / 2, altura),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')

    adicionar_rotulos(barras1)
    adicionar_rotulos(barras2)

    plt.tight_layout()
    plt.show()

# Exemplo de uso:
analisar_grupo_cliente(codigo_grupo_cliente='1958')
